In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3a2mext/3A2M_EXTENDED.csv


In [10]:
# Install all required packages
!pip install transformers datasets accelerate rouge-score nltk -q

# Download NLTK data for BLEU score
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
# Recipe Generation using GPT-2 - Data Preparation
# This script prepares the recipe dataset for fine-tuning

import pandas as pd
import json
import os
import ast
from datasets import Dataset
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

# Dataset path
dataset_path = '/kaggle/input/3a2mext/3A2M_EXTENDED.csv'

# Load the dataset
print("Loading dataset...")
df = pd.read_csv(dataset_path)

# Display basic info
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
print(df.head())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Clean the data
# The dataset uses 'NER' for ingredients
df = df.dropna(subset=['title', 'NER', 'directions'])
print(f"\nDataset shape after removing nulls: {df.shape}")

# Function to clean and parse ingredients
def parse_ingredients(ing_str):
    """Convert string representation of list to actual list"""
    try:
        # Parse the string as a Python list
        ing_list = ast.literal_eval(ing_str)
        # Join ingredients with commas
        return ', '.join(ing_list)
    except:
        return ing_str

# Function to clean and parse directions
def parse_directions(dir_str):
    """Convert string representation of list to actual list"""
    try:
        # Parse the string as a Python list
        dir_list = ast.literal_eval(dir_str)
        # Join directions with numbered steps
        steps = [f"{i+1}. {step}" for i, step in enumerate(dir_list)]
        return ' '.join(steps)
    except:
        return dir_str

# Clean title (remove \t if present)
df['title'] = df['title'].str.strip()

# Parse ingredients and directions
print("\nParsing ingredients and directions...")
df['ingredients_clean'] = df['NER'].apply(parse_ingredients)
df['directions_clean'] = df['directions'].apply(parse_directions)

# Function to format recipe into text format for GPT-2
def format_recipe(row):
    """
    Format: 
    Recipe: [title]
    Ingredients: [ingredients]
    Instructions: [directions]
    """
    text = f"Recipe: {row['title']}\n"
    text += f"Ingredients: {row['ingredients_clean']}\n"
    text += f"Instructions: {row['directions_clean']}\n"
    text += "<|endoftext|>"  # Special token to mark end
    return text

# Create formatted text column
print("Formatting recipes...")
df['text'] = df.apply(format_recipe, axis=1)

# Show example
print("\nExample formatted recipe:")
print("="*60)
print(df['text'].iloc[0][:800])
print("="*60)

# Remove extremely long or short recipes
df['text_length'] = df['text'].apply(len)
print(f"\nText length statistics:")
print(df['text_length'].describe())

# Filter recipes between 100 and 2000 characters
df = df[(df['text_length'] > 100) & (df['text_length'] < 2000)]
print(f"Dataset shape after length filtering: {df.shape}")

# Use a sample for faster training (student project)
# Adjust this based on your computational resources
sample_size = 10000  # Start with 10k, reduce if needed

if len(df) > sample_size:
    df = df.sample(n=sample_size, random_state=42)
    print(f"\nUsing {sample_size} samples for training")
else:
    print(f"\nUsing all {len(df)} samples")

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"\nTraining samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# Save processed data
train_df[['text']].to_csv('train_recipes.csv', index=False)
val_df[['text']].to_csv('val_recipes.csv', index=False)

# Also save a few examples for reference
examples_df = train_df[['title', 'ingredients_clean', 'directions_clean', 'text']].head(10)
examples_df.to_csv('recipe_examples.csv', index=False)

print("\n" + "="*60)
print("Data preparation complete!")
print("="*60)
print("\nSaved files:")
print("- train_recipes.csv")
print("- val_recipes.csv")
print("- recipe_examples.csv")

print("\nDataset statistics:")
print(f"Total recipes processed: {len(df)}")
print(f"Training set: {len(train_df)}")
print(f"Validation set: {len(val_df)}")
print(f"Average recipe length: {df['text_length'].mean():.0f} characters")

Loading dataset...
Dataset shape: (2231143, 6)
Columns: ['title', 'NER', 'Extended_NER', 'genre', 'label', 'directions']

First few rows:
                                         title  \
0                 \t Arugula Pomegranate Salad   
1               \t Black Bean And Turkey Chili   
2               \t Finger Lickin' Tofu Nuggets   
3  \t Jerk Beef Stew With Carrots And Tomatoes   
4                \t Pomegranate Couscous Salad   

                                                 NER  \
0  ["baby spinach", "baby arugula", "pomegranate ...   
1  ["olive oil", "yellow onion", "garlic", "groun...   
2  ["extra firm", "almond flour", "nutritional ye...   
3  ["olive oil", "boneless beef chuck", "onion", ...   
4  ["pomegranate arils", "whole wheat couscous", ...   

                                        Extended_NER       genre  label  \
0  ['alfalfa sprouts', 'baby spinach', 'baby arug...  vegetables      4   
1  ['one', 'yellow onion', 'tomato paste', 'about...       sides      8   

In [8]:
# GPT-2 Fine-tuning for Recipe Generation
# Training Script

import pandas as pd
import torch
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer, 
    Trainer, 
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load tokenizer and model
print("\nLoading GPT-2 model and tokenizer...")
model_name = "gpt2"  # You can also try "gpt2-medium" if you have more resources
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

print(f"Model loaded: {model_name}")
print(f"Vocab size: {len(tokenizer)}")

# Load prepared data
print("\nLoading prepared datasets...")
train_df = pd.read_csv('train_recipes.csv')
val_df = pd.read_csv('val_recipes.csv')

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization function
def tokenize_function(examples):
    """Tokenize the text data"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=512,  # Adjust based on your recipes length
        padding='max_length'
    )

# Tokenize datasets
print("\nTokenizing datasets...")
tokenized_train = train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=['text']
)
tokenized_val = val_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=['text']
)

print("Tokenization complete!")

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 uses causal language modeling, not masked
)

# Training arguments - adjusted for student project on Kaggle
training_args = TrainingArguments(
    output_dir='./recipe-gpt2-finetuned',
    overwrite_output_dir=True,
    num_train_epochs=3,  # Start with 3, can increase if needed
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,  # Only keep 2 best checkpoints to save space
    prediction_loss_only=True,
    report_to='none',  # Disable wandb/tensorboard for simplicity
    fp16=True if device == 'cuda' else False,  # Use mixed precision on GPU
)

print("\nTraining Arguments:")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Start training
print("\n" + "="*50)
print("Starting training...")
print("="*50 + "\n")

trainer.train()

# Save the final model
print("\nSaving model...")
trainer.save_model('./recipe-gpt2-final')
tokenizer.save_pretrained('./recipe-gpt2-final')

print("\n" + "="*50)
print("Training complete!")
print("Model saved to: ./recipe-gpt2-final")
print("="*50)

2025-11-03 21:20:13.209283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762204813.634726      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762204813.756690      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda

Loading GPT-2 model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Tokenization complete!

Training Arguments:
Epochs: 3
Batch size: 4
Learning rate: 5e-05

Starting training...



`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,1.244200,2.316139
1000,1.201700,2.233753
1500,1.133100,2.186581
2000,1.089900,2.152513
2500,1.051700,2.129192
3000,1.071300,2.121942


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Saving model...

Training complete!
Model saved to: ./recipe-gpt2-final


In [12]:
# Testing and Evaluation Script for Fine-tuned Recipe GPT-2

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import pandas as pd
import numpy as np

# Load the fine-tuned model
print("Loading fine-tuned model...")
model_path = './recipe-gpt2-final'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

print(f"Model loaded on {device}")

# Function to generate recipe
def generate_recipe(prompt, max_length=300, temperature=0.8, top_p=0.9):
    """
    Generate a recipe given a prompt
    
    Args:
        prompt: Starting text (e.g., "Recipe: Chocolate Cake")
        max_length: Maximum tokens to generate
        temperature: Controls randomness (higher = more random)
        top_p: Nucleus sampling parameter
    """
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1
        )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Test examples
print("\n" + "="*60)
print("TESTING RECIPE GENERATION")
print("="*60)

# Example 1: Generate from dish name
print("\n--- Example 1: Chocolate Cake ---")
prompt1 = "Recipe: Chocolate Cake\nIngredients:"
recipe1 = generate_recipe(prompt1)
print(recipe1)

# Example 2: Generate from ingredients
print("\n--- Example 2: Chicken with vegetables ---")
prompt2 = "Recipe: Grilled Chicken\nIngredients:"
recipe2 = generate_recipe(prompt2)
print(recipe2)

# Example 3: Another dish
print("\n--- Example 3: Pasta ---")
prompt3 = "Recipe: Spaghetti Carbonara\nIngredients:"
recipe3 = generate_recipe(prompt3)
print(recipe3)

# Example 4: Custom ingredients
print("\n--- Example 4: From ingredients ---")
prompt4 = "Recipe: Vegetable Stir Fry\nIngredients: broccoli, carrots, soy sauce, garlic\nInstructions:"
recipe4 = generate_recipe(prompt4, max_length=200)
print(recipe4)

# Evaluation metrics
print("\n" + "="*60)
print("EVALUATION METRICS")
print("="*60)

# Load validation data for evaluation
val_df = pd.read_csv('val_recipes.csv')
sample_recipes = val_df.sample(n=10, random_state=42)

# Initialize rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
bleu_scores = []
rouge_scores = []

print("\nCalculating BLEU and ROUGE scores on validation samples...")
print("This may take a few minutes...\n")

for idx, (i, row) in enumerate(sample_recipes.iterrows()):
    print(f"Evaluating sample {idx+1}/10...", end='\r')
    
    # Extract prompt from original recipe
    original = row['text']
    
    # Get first line as prompt
    lines = original.split('\n')
    if len(lines) > 0:
        prompt = lines[0] + "\nIngredients:"
    else:
        continue
    
    # Generate recipe
    generated = generate_recipe(prompt, max_length=300, temperature=0.7)
    
    # Calculate BLEU score
    reference = [original.split()]
    candidate = generated.split()
    smoothie = SmoothingFunction().method4
    bleu = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    bleu_scores.append(bleu)
    
    # Calculate ROUGE scores using rouge_score package
    try:
        scores = scorer.score(original, generated)
        rouge_scores.append(scores['rougeL'].fmeasure)
    except:
        pass

print("\n")  # New line after progress

# Print results
if bleu_scores:
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"\nAverage BLEU Score: {avg_bleu:.4f}")

if rouge_scores:
    avg_rouge = sum(rouge_scores) / len(rouge_scores)
    print(f"Average ROUGE-L F1 Score: {avg_rouge:.4f}")

print("\n" + "="*60)
print("Evaluation complete!")
print("="*60)

# Save sample generations
print("\nSaving sample generations...")
examples = {
    'Prompt': [
        "Recipe: Chocolate Cake",
        "Recipe: Grilled Chicken", 
        "Recipe: Spaghetti Carbonara",
        "Recipe: Vegetable Stir Fry"
    ],
    'Generated Recipe': [recipe1, recipe2, recipe3, recipe4]
}

results_df = pd.DataFrame(examples)
results_df.to_csv('generated_recipes_examples.csv', index=False)
print("Saved to: generated_recipes_examples.csv")

Loading fine-tuned model...
Model loaded on cuda

TESTING RECIPE GENERATION

--- Example 1: Chocolate Cake ---
Recipe: Chocolate Cake
Ingredients: shortening, sugar, baking soda, salt, cake flour, brown sugar, butter, egg whites, chocolate, sugar, vanilla, eggs, milk, baking powder, cinnamon, nutmeg, buttermilk, coconut, nuts
Instructions: 1. In a bowl, whisk together shortening, sugar, baking soda, salt, baking soda, and nutmeg. 2. Add eggs, milk, baking powder, cinnamon, nutmeg, buttermilk, coconut, nuts, and vanilla. 3. Mix well. 4. Stir in nuts. 5. Fold in remaining ingredients. 6. Bake at 350° for 1 hour.
Nutritional Information: Calories: Fat: 2.5 g, cholesterol: 2.5 g, sodium: 1 g, cholesterol: 3 g, saturated fat: 1 g, cholesterol: 2.5 g, cholesterol: 4.5 g
Ingredients: flour, cocoa, milk, cocoa powder, baking powder, cinnamon, nutmeg, buttermilk, coconut, nuts, coconut, chocolate, sugar, vanilla, eggs, milk, baking powder, cinnamon, nutmeg, buttermilk, coconut, nuts, coconut, a

In [13]:
# Improved Recipe Generation Script
# This version adds better generation controls and post-processing

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

# Load the fine-tuned model
print("Loading fine-tuned model...")
model_path = './recipe-gpt2-final'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

print(f"Model loaded on {device}")

# Improved generation function
def generate_recipe_improved(prompt, max_length=400, temperature=0.7, 
                            top_p=0.9, top_k=50, repetition_penalty=1.2):
    """
    Generate recipe with improved parameters to reduce repetition
    
    Args:
        prompt: Starting text
        max_length: Maximum tokens
        temperature: Randomness (0.7 = more focused, 0.9 = more creative)
        top_p: Nucleus sampling
        top_k: Top-k sampling
        repetition_penalty: Penalty for repeating tokens (>1.0 reduces repetition)
    """
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # Create attention mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            no_repeat_ngram_size=3  # Prevent 3-gram repetition
        )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Post-process: Stop at natural ending
    # Look for second occurrence of "Recipe:" or end token
    lines = generated_text.split('\n')
    clean_lines = []
    recipe_count = 0
    
    for line in lines:
        if line.strip().startswith('Recipe:'):
            recipe_count += 1
            if recipe_count > 1:  # Stop at second recipe
                break
        clean_lines.append(line)
    
    return '\n'.join(clean_lines)

# Test with different parameter settings
print("\n" + "="*60)
print("TESTING IMPROVED GENERATION")
print("="*60)

# Example 1: More focused (lower temperature)
print("\n--- Example 1: Chocolate Cake (Focused) ---")
prompt1 = "Recipe: Chocolate Cake\nIngredients:"
recipe1 = generate_recipe_improved(prompt1, temperature=0.7, repetition_penalty=1.3)
print(recipe1)

# Example 2: More creative (higher temperature)
print("\n--- Example 2: Thai Curry (Creative) ---")
prompt2 = "Recipe: Thai Green Curry\nIngredients:"
recipe2 = generate_recipe_improved(prompt2, temperature=0.85, repetition_penalty=1.2)
print(recipe2)

# Example 3: From ingredients
print("\n--- Example 3: From Ingredients ---")
prompt3 = "Recipe: Healthy Breakfast Bowl\nIngredients: oats, banana, honey, almonds, berries\nInstructions:"
recipe3 = generate_recipe_improved(prompt3, temperature=0.75, max_length=300)
print(recipe3)

# Generate multiple variations
print("\n" + "="*60)
print("GENERATING VARIATIONS")
print("="*60)

def generate_variations(prompt, n=3):
    """Generate n variations of the same recipe"""
    print(f"\nGenerating {n} variations for: {prompt}\n")
    variations = []
    
    for i in range(n):
        print(f"Variation {i+1}:")
        print("-" * 40)
        # Use different temperatures for variety
        temp = 0.7 + (i * 0.1)
        recipe = generate_recipe_improved(prompt, temperature=temp, repetition_penalty=1.2)
        variations.append(recipe)
        print(recipe)
        print()
    
    return variations

# Test variations
prompt = "Recipe: Vegetable Soup\nIngredients:"
variations = generate_variations(prompt, n=3)

print("\n" + "="*60)
print("QUALITY ANALYSIS")
print("="*60)

# Simple quality metrics
def analyze_quality(recipe_text):
    """Analyze basic quality metrics"""
    lines = recipe_text.split('\n')
    
    # Count sections
    has_ingredients = any('Ingredients:' in line for line in lines)
    has_instructions = any('Instructions:' in line for line in lines)
    
    # Count ingredients and steps
    ingredient_count = len([l for l in lines if l.strip() and 'Ingredients:' not in l and 'Instructions:' not in l and 'Recipe:' not in l and not l[0].isdigit()])
    step_count = len([l for l in lines if l.strip() and l[0].isdigit()])
    
    # Check repetition
    words = recipe_text.lower().split()
    unique_ratio = len(set(words)) / len(words) if words else 0
    
    return {
        'has_ingredients': has_ingredients,
        'has_instructions': has_instructions,
        'ingredient_count': ingredient_count,
        'step_count': step_count,
        'unique_word_ratio': unique_ratio,
        'total_length': len(recipe_text)
    }

# Analyze examples
print("\nAnalyzing generated recipes:\n")
for i, recipe in enumerate([recipe1, recipe2, recipe3], 1):
    print(f"Recipe {i} Analysis:")
    metrics = analyze_quality(recipe)
    print(f"  ✓ Has Ingredients: {metrics['has_ingredients']}")
    print(f"  ✓ Has Instructions: {metrics['has_instructions']}")
    print(f"  - Estimated ingredients: {metrics['ingredient_count']}")
    print(f"  - Estimated steps: {metrics['step_count']}")
    print(f"  - Unique word ratio: {metrics['unique_word_ratio']:.2f}")
    print(f"  - Length: {metrics['total_length']} characters")
    print()

print("="*60)
print("Generation complete!")
print("="*60)

# Save improved examples
examples_data = {
    'Recipe_Name': ['Chocolate Cake', 'Thai Green Curry', 'Healthy Breakfast Bowl'],
    'Generated_Text': [recipe1, recipe2, recipe3],
    'Temperature': [0.7, 0.85, 0.75]
}

df_examples = pd.DataFrame(examples_data)
df_examples.to_csv('improved_recipe_examples.csv', index=False)
print("\n✅ Saved improved examples to: improved_recipe_examples.csv")

Loading fine-tuned model...
Model loaded on cuda

TESTING IMPROVED GENERATION

--- Example 1: Chocolate Cake (Focused) ---
Recipe: Chocolate Cake
Ingredients: cake mix, sugar and cocoa powder; eggs. Beat egg yolks with vanilla and 1/2 cup flour until light but not dry. Add powdered sugar to mixture mixing well. Pour batter into greased 9 x 13-inch pan. Bake at 350\u00b0 for 45 minutes or till done when cool enough that cake can be used as a topping. Cool in pans on wire rack 10 minutes before cutting into squares. Makes 8 dozen (9" x 12") cookies.
Instructions For baking: Mix together the first 4 ingredients then add them to pudding mix while still warm. In a small bowl combine cream cheese, lemon juice extract from pineapple chunks, chocolate chips & marshmallows. Stir gently all over thoroughly. Spread evenly around edges of pan top. Sprinkle with chopped pecans. Top with remaining whipped creme. Let stand 20 minutes covered before serving. Serves 6.
Ingredients : graham cracker crum

In [15]:
# Create a zip file of the model
import shutil
shutil.make_archive('recipe-gpt2-final', 'zip', './recipe-gpt2-final')

# Download - ADD THIS LINE
from IPython.display import FileLink
FileLink('recipe-gpt2-final.zip')

/kaggle/working/recipe-gpt2-final.zip